In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from functools import partial
from tensorflow import keras
import sys
sys.path.append("..")
from train_models import train_merged, probe_dir
from data_prep import norm_image
from evaluate_models import plot_cm, process_labels, calc_precision_recall, calc_f1
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import time

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


ImportError: cannot import name 'train_merged' from 'train_models' (/tf/notebooks/src/Galaxy_Networks/../train_models.py)

In [ ]:
X = np.load('../../data/galaxy_X.npy')
y = np.load('../../data/galaxy_y.npy')
y_aux = np.load('../../data/galaxy_y_aux.npy')

In [ ]:
#Set seeds to be able to reproduce network
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Split dataset
X_tmp_train, X_test1, y_aux_tmp_train, y_aux_test1, y_tmp_train, y_test = train_test_split(X, y_aux, y, test_size=0.1, train_size=0.9, random_state=42, shuffle=True, stratify=y)
X_train1, X_val1, y_aux_train1, y_aux_val1, y_train, y_val = train_test_split(X_tmp_train, y_aux_tmp_train, y_tmp_train, test_size=0.11, train_size=0.89, random_state=42, shuffle=True, stratify=y_tmp_train)

In [ ]:
# Apply normalization to images
def normalize_images(X):
    return np.array(list(map(normalize_image, X)))
    
def normalize_image(img):
    bot = np.min(img)
    top = np.max(img)
    norm = (img - bot)/(top - bot)
    return norm

X_train = normalize_images(X_train1)
X_val = normalize_images(X_val1)
X_test = normalize_images(X_test1)

X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
# Normalize features to allow for sigmoid activation
def normalize_targets(y):
    bot = np.min(y, axis=0)
    top = np.max(y, axis=0)
    norm = (y - bot)/(top - bot)
    return norm

y_aux_train = normalize_targets(y_aux_train1)
y_aux_val = normalize_targets(y_aux_val1)
y_aux_test = normalize_targets(y_aux_test1)

# Even weights

In [ ]:
# Evenly weighted heads
runs = 20
total_loss = 0
total_lbl_entropy = 0
total_bent_entropy = 0
total_fr_mse = 0
total_cores_mse = 0
total_size_mse = 0
total_lbl_acc = 0
total_bent_acc = 0
total_fr_mae = 0
total_cores_mae = 0
total_size_mae = 0
elapsed = 0

# Execute a few runs of training the network
for run in range(1,runs+1):
    start = time.time()
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = train_merged(X_train, y_train, y_aux_train,
                                          X_val, y_val, y_aux_val, X_test, y_test, y_aux_test, 0.5, 0.5, run)
    keras.backend.clear_session()
    end = time.time()
    elapsed += (end - start)
    total_loss += tmp_loss
    total_lbl_entropy += tmp_lbl_entropy
    total_bent_entropy += tmp_bent_entropy
    total_fr_mse += tmp_fr_mse
    total_cores_mse += tmp_cores_mse
    total_size_mse += tmp_size_mse
    total_lbl_acc += tmp_lbl_acc
    total_bent_acc += tmp_bent_acc
    total_fr_mae += tmp_fr_mae
    total_cores_mae += tmp_cores_mae
    total_size_mae += tmp_size_mae
avg_loss = total_loss/runs
avg_lbl_entropy = total_lbl_entropy/runs
avg_bent_entropy = total_bent_entropy/runs
avg_fr_mse = total_fr_mse/runs
avg_cores_mse = total_cores_mse/runs
avg_size_mse = total_size_mse/runs
avg_lbl_acc = total_lbl_acc/runs
avg_bent_acc = total_bent_acc/runs
avg_fr_mae = total_fr_mae/runs
avg_cores_mae = total_cores_mae/runs
avg_size_mae = total_size_mae/runs
avg_elapsed = elapsed/runs

In [ ]:
runs = 20
epochs_log = []
for run in range(1,runs+1):
    ea = EventAccumulator(f'../../lr_logs/merged_0.5_0.5_run{run}/train')
    ea.Reload()
    ct_loss, epochs_loss, loss = zip(*ea.Tensors('epoch_loss'))
    epochs_loss = np.asarray(epochs_loss)
    epochs_log.append(epochs_loss[-1] + 1)

epochs_log = np.array(epochs_log)
avg_epochs = np.mean(epochs_log)
min_idx = np.argmin(epochs_log) + 1
min_epochs = epochs_log[min_idx - 1]
max_idx = np.argmax(epochs_log) + 1
max_epochs = epochs_log[max_idx - 1]

In [ ]:
avg_mse = (avg_fr_mse + avg_cores_mse + avg_size_mse)/3
avg_mae = (avg_fr_mae + avg_cores_mae + avg_size_mae)/3
tmp = np.array([avg_lbl_entropy, avg_bent_entropy, avg_fr_mse, avg_cores_mse, avg_size_mse, avg_lbl_acc, avg_bent_acc, avg_fr_mae, avg_cores_mae, avg_size_mae, avg_mse, avg_mae, avg_loss, avg_elapsed, avg_epochs, max_epochs, max_idx, min_epochs, min_idx])

probe_dir('../../results/')
np.save('../../results/merged_even_results.npy', tmp)
tmp = np.load('../../results/merged_even_results.npy')

In [ ]:
print(f"Average classification entropy: {tmp[0]}")
print(f"Average bent entropy: {tmp[1]}")
print(f"Average FR ratio MSE: {tmp[2]}")
print(f"Average core count MSE: {tmp[3]}")
print(f"Average core ratio MSE: {tmp[4]}")
print(f"Average classification accuracy: {tmp[5]}")
print(f"Average bent accuracy: {tmp[6]}")
print(f"Average FR ratio MAE: {tmp[7]}")
print(f"Average core count MAE: {tmp[8]}")
print(f"Average core ratio MAE: {tmp[9]}")
print(f"Average overall MSE: {tmp[10]}")
print(f"Average overall MAE: {tmp[11]}")
print(f"Average loss: {tmp[12]}")
print(f"Average time taken: {tmp[13]}")
print(f"Average epochs taken: {tmp[14]}")
print(f"Maximum number of epochs taken was {tmp[15]} at run {tmp[16]}")
print(f"Minimum number of epochs taken was {tmp[17]} at run {tmp[18]}")

In [ ]:
def extract_class_pred(preds):
    return preds[0]

In [ ]:
bent_precs, bent_recalls, bent_f1s = [], [], []
comp_precs, comp_recalls, comp_f1s = [], [], []
fri_precs, fri_recalls, fri_f1s = [], [], []
frii_precs, frii_recalls, frii_f1s = [], [], []
macro_f1s = []
average_cm = np.zeros((4, 4))
runs = 20
for run in range(1,runs+1):
    best_model = keras.models.load_model(f"../../models/merged_0.5_0.5_model{run}.h5")
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    average_cm += cm
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_precs.append(bent_prec)
    bent_recalls.append(bent_recall)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_precs.append(comp_prec)
    comp_recalls.append(comp_recall)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_precs.append(fri_prec)
    fri_recalls.append(fri_recall)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_precs.append(frii_prec)
    frii_recalls.append(frii_recall)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)

average_cm = average_cm/runs
print(f'Average bent precision: {np.mean(bent_precs)}')
print(f'Average bent recall: {np.mean(bent_recalls)}')
print(f'Average bent F1: {np.mean(bent_f1s)}')
print(f'Average comp precision: {np.mean(comp_precs)}')
print(f'Average comp recall: {np.mean(comp_recalls)}')
print(f'Average comp F1: {np.mean(comp_f1s)}')
print(f'Average FRI precision: {np.mean(fri_precs)}')
print(f'Average FRI recall: {np.mean(fri_recalls)}')
print(f'Average FRI F1: {np.mean(fri_f1s)}')
print(f'Average FRII precision: {np.mean(frii_precs)}')
print(f'Average FRII recall: {np.mean(frii_recalls)}')
print(f'Average FRII F1: {np.mean(frii_f1s)}')
print(f'Average Macro F1: {np.mean(macro_f1s)}')
plot_cm(average_cm)

# Heavier auxiliary weight

In [ ]:
runs = 20
total_loss = 0
total_lbl_entropy = 0
total_bent_entropy = 0
total_fr_mse = 0
total_cores_mse = 0
total_size_mse = 0
total_lbl_acc = 0
total_bent_acc = 0
total_fr_mae = 0
total_cores_mae = 0
total_size_mae = 0
elapsed = 0

# Execute a few runs of training the network
for run in range(1,runs+1):
    start = time.time()
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = train_merged(X_train, y_train, y_aux_train,
                                          X_val, y_val, y_aux_val, X_test, y_test, y_aux_test, 0.25, 0.75, run)
    keras.backend.clear_session()
    end = time.time()
    elapsed += (end - start)
    total_loss += tmp_loss
    total_lbl_entropy += tmp_lbl_entropy
    total_bent_entropy += tmp_bent_entropy
    total_fr_mse += tmp_fr_mse
    total_cores_mse += tmp_cores_mse
    total_size_mse += tmp_size_mse
    total_lbl_acc += tmp_lbl_acc
    total_bent_acc += tmp_bent_acc
    total_fr_mae += tmp_fr_mae
    total_cores_mae += tmp_cores_mae
    total_size_mae += tmp_size_mae
avg_loss = total_loss/runs
avg_lbl_entropy = total_lbl_entropy/runs
avg_bent_entropy = total_bent_entropy/runs
avg_fr_mse = total_fr_mse/runs
avg_cores_mse = total_cores_mse/runs
avg_size_mse = total_size_mse/runs
avg_lbl_acc = total_lbl_acc/runs
avg_bent_acc = total_bent_acc/runs
avg_fr_mae = total_fr_mae/runs
avg_cores_mae = total_cores_mae/runs
avg_size_mae = total_size_mae/runs
avg_elapsed = elapsed/runs

In [ ]:
runs = 20
epochs_log = []
for run in range(1,runs+1):
    ea = EventAccumulator(f'../../lr_logs/merged_0.25_0.75_run{run}/train')
    ea.Reload()
    ct_loss, epochs_loss, loss = zip(*ea.Tensors('epoch_loss'))
    epochs_loss = np.asarray(epochs_loss)
    epochs_log.append(epochs_loss[-1] + 1)

epochs_log = np.array(epochs_log)
avg_epochs = np.mean(epochs_log)
min_idx = np.argmin(epochs_log) + 1
min_epochs = epochs_log[min_idx - 1]
max_idx = np.argmax(epochs_log) + 1
max_epochs = epochs_log[max_idx - 1]

In [ ]:
avg_mse = (avg_fr_mse + avg_cores_mse + avg_size_mse)/3
avg_mae = (avg_fr_mae + avg_cores_mae + avg_size_mae)/3
tmp = np.array([avg_lbl_entropy, avg_bent_entropy, avg_fr_mse, avg_cores_mse, avg_size_mse, avg_lbl_acc, avg_bent_acc, avg_fr_mae, avg_cores_mae, avg_size_mae, avg_mse, avg_mae, avg_loss, avg_elapsed, avg_epochs, max_epochs, max_idx, min_epochs, min_idx])
probe_dir('../../results/')
np.save('../../results/merged_aux_results.npy', tmp)
tmp = np.load('../../results/merged_aux_results.npy')

In [ ]:
print(f"Average classification entropy: {tmp[0]}")
print(f"Average bent entropy: {tmp[1]}")
print(f"Average FR ratio MSE: {tmp[2]}")
print(f"Average core count MSE: {tmp[3]}")
print(f"Average core ratio MSE: {tmp[4]}")
print(f"Average classification accuracy: {tmp[5]}")
print(f"Average bent accuracy: {tmp[6]}")
print(f"Average FR ratio MAE: {tmp[7]}")
print(f"Average core count MAE: {tmp[8]}")
print(f"Average core ratio MAE: {tmp[9]}")
print(f"Average overall MSE: {tmp[10]}")
print(f"Average overall MAE: {tmp[11]}")
print(f"Average loss: {tmp[12]}")
print(f"Average time taken: {tmp[13]}")
print(f"Average epochs taken: {tmp[14]}")
print(f"Maximum number of epochs taken was {tmp[15]} at run {tmp[16]}")
print(f"Minimum number of epochs taken was {tmp[17]} at run {tmp[18]}")

In [ ]:
bent_precs, bent_recalls, bent_f1s = [], [], []
comp_precs, comp_recalls, comp_f1s = [], [], []
fri_precs, fri_recalls, fri_f1s = [], [], []
frii_precs, frii_recalls, frii_f1s = [], [], []
macro_f1s = []
average_cm = np.zeros((4, 4))
runs = 20
for run in range(1,runs+1):
    best_model = keras.models.load_model(f"../../models/merged_0.25_0.75_model{run}.h5")
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    average_cm += cm
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_precs.append(bent_prec)
    bent_recalls.append(bent_recall)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_precs.append(comp_prec)
    comp_recalls.append(comp_recall)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_precs.append(fri_prec)
    fri_recalls.append(fri_recall)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_precs.append(frii_prec)
    frii_recalls.append(frii_recall)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)

average_cm = average_cm/runs
print(f'Average bent precision: {np.mean(bent_precs)}')
print(f'Average bent recall: {np.mean(bent_recalls)}')
print(f'Average bent F1: {np.mean(bent_f1s)}')
print(f'Average comp precision: {np.mean(comp_precs)}')
print(f'Average comp recall: {np.mean(comp_recalls)}')
print(f'Average comp F1: {np.mean(comp_f1s)}')
print(f'Average FRI precision: {np.mean(fri_precs)}')
print(f'Average FRI recall: {np.mean(fri_recalls)}')
print(f'Average FRI F1: {np.mean(fri_f1s)}')
print(f'Average FRII precision: {np.mean(frii_precs)}')
print(f'Average FRII recall: {np.mean(frii_recalls)}')
print(f'Average FRII F1: {np.mean(frii_f1s)}')
print(f'Average Macro F1: {np.mean(macro_f1s)}')
plot_cm(average_cm)

# Heavier main weight

In [ ]:
# Heavier main head
runs = 20
total_loss = 0
total_lbl_entropy = 0
total_bent_entropy = 0
total_fr_mse = 0
total_cores_mse = 0
total_size_mse = 0
total_lbl_acc = 0
total_bent_acc = 0
total_fr_mae = 0
total_cores_mae = 0
total_size_mae = 0
elapsed = 0

# Execute a few runs of training the network
for run in range(1,runs+1):
    start = time.time()
    tmp_loss, tmp_lbl_entropy, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_lbl_acc, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = train_galaxy_merged(X_train, y_train, y_aux_train,
                                          X_val, y_val, y_aux_val, X_test, y_test, y_aux_test, 0.75, 0.25, run)
    keras.backend.clear_session()
    end = time.time()
    elapsed += (end - start)
    total_loss += tmp_loss
    total_lbl_entropy += tmp_lbl_entropy
    total_bent_entropy += tmp_bent_entropy
    total_fr_mse += tmp_fr_mse
    total_cores_mse += tmp_cores_mse
    total_size_mse += tmp_size_mse
    total_lbl_acc += tmp_lbl_acc
    total_bent_acc += tmp_bent_acc
    total_fr_mae += tmp_fr_mae
    total_cores_mae += tmp_cores_mae
    total_size_mae += tmp_size_mae
avg_loss = total_loss/runs
avg_lbl_entropy = total_lbl_entropy/runs
avg_bent_entropy = total_bent_entropy/runs
avg_fr_mse = total_fr_mse/runs
avg_cores_mse = total_cores_mse/runs
avg_size_mse = total_size_mse/runs
avg_lbl_acc = total_lbl_acc/runs
avg_bent_acc = total_bent_acc/runs
avg_fr_mae = total_fr_mae/runs
avg_cores_mae = total_cores_mae/runs
avg_size_mae = total_size_mae/runs
avg_elapsed = elapsed/runs

In [ ]:
runs = 20
epochs_log = []
for run in range(1,runs+1):
    ea = EventAccumulator(f'../../lr_logs/merged_0.75_0.25_run{run}/train')
    ea.Reload()
    ct_loss, epochs_loss, loss = zip(*ea.Tensors('epoch_loss'))
    epochs_loss = np.asarray(epochs_loss)
    epochs_log.append(epochs_loss[-1] + 1)

epochs_log = np.array(epochs_log)
avg_epochs = np.mean(epochs_log)
min_idx = np.argmin(epochs_log) + 1
min_epochs = epochs_log[min_idx - 1]
max_idx = np.argmax(epochs_log) + 1
max_epochs = epochs_log[max_idx - 1]

In [ ]:
avg_mse = (avg_fr_mse + avg_cores_mse + avg_size_mse)/3
avg_mae = (avg_fr_mae + avg_cores_mae + avg_size_mae)/3
tmp = np.array([avg_lbl_entropy, avg_bent_entropy, avg_fr_mse, avg_cores_mse, avg_size_mse, avg_lbl_acc, avg_bent_acc, avg_fr_mae, avg_cores_mae, avg_size_mae, avg_mse, avg_mae, avg_loss, avg_elapsed, avg_epochs, max_epochs, max_idx, min_epochs, min_idx])
probe_dir('../../results/')
np.save('../../results/merged_main_results.npy', tmp)
tmp = np.load('../../results/merged_main_results.npy')

In [ ]:
print(f"Average classification entropy: {tmp[0]}")
print(f"Average bent entropy: {tmp[1]}")
print(f"Average FR ratio MSE: {tmp[2]}")
print(f"Average core count MSE: {tmp[3]}")
print(f"Average core ratio MSE: {tmp[4]}")
print(f"Average classification accuracy: {tmp[5]}")
print(f"Average bent accuracy: {tmp[6]}")
print(f"Average FR ratio MAE: {tmp[7]}")
print(f"Average core count MAE: {tmp[8]}")
print(f"Average core ratio MAE: {tmp[9]}")
print(f"Average overall MSE: {tmp[10]}")
print(f"Average overall MAE: {tmp[11]}")
print(f"Average loss: {tmp[12]}")
print(f"Average time taken: {tmp[13]}")
print(f"Average epochs taken: {tmp[14]}")
print(f"Maximum number of epochs taken was {tmp[15]} at run {tmp[16]}")
print(f"Minimum number of epochs taken was {tmp[17]} at run {tmp[18]}")

In [ ]:
bent_precs, bent_recalls, bent_f1s = [], [], []
comp_precs, comp_recalls, comp_f1s = [], [], []
fri_precs, fri_recalls, fri_f1s = [], [], []
frii_precs, frii_recalls, frii_f1s = [], [], []
macro_f1s = []
average_cm = np.zeros((4, 4))
runs = 20
for run in range(1,runs+1):
    best_model = keras.models.load_model(f"../../models/merged_0.75_0.25_model{run}.h5")
    test_pred = extract_class_pred(best_model.predict(X_test))
    new_test_true, new_test_pred = process_labels(y_test, test_pred)
    cm = confusion_matrix(new_test_true, new_test_pred)
    average_cm += cm
    bent_prec, bent_recall = calc_precision_recall(cm, 0)
    bent_precs.append(bent_prec)
    bent_recalls.append(bent_recall)
    bent_f1s.append(calc_f1(bent_prec, bent_recall))
    comp_prec, comp_recall = calc_precision_recall(cm, 1)
    comp_precs.append(comp_prec)
    comp_recalls.append(comp_recall)
    comp_f1s.append(calc_f1(comp_prec, comp_recall))
    fri_prec, fri_recall = calc_precision_recall(cm, 2)
    fri_precs.append(fri_prec)
    fri_recalls.append(fri_recall)
    fri_f1s.append(calc_f1(fri_prec, fri_recall))
    frii_prec, frii_recall = calc_precision_recall(cm, 3)
    frii_precs.append(frii_prec)
    frii_recalls.append(frii_recall)
    frii_f1s.append(calc_f1(frii_prec, frii_recall))
    macro_f1s.append((bent_f1s[-1] + comp_f1s[-1] + fri_f1s[-1] + frii_f1s[-1])/4)

average_cm = average_cm/runs
print(f'Average bent precision: {np.mean(bent_precs)}')
print(f'Average bent recall: {np.mean(bent_recalls)}')
print(f'Average bent F1: {np.mean(bent_f1s)}')
print(f'Average comp precision: {np.mean(comp_precs)}')
print(f'Average comp recall: {np.mean(comp_recalls)}')
print(f'Average comp F1: {np.mean(comp_f1s)}')
print(f'Average FRI precision: {np.mean(fri_precs)}')
print(f'Average FRI recall: {np.mean(fri_recalls)}')
print(f'Average FRI F1: {np.mean(fri_f1s)}')
print(f'Average FRII precision: {np.mean(frii_precs)}')
print(f'Average FRII recall: {np.mean(frii_recalls)}')
print(f'Average FRII F1: {np.mean(frii_f1s)}')
print(f'Average Macro F1: {np.mean(macro_f1s)}')
plot_cm(average_cm)